### sqlalchemy 라이브러리 호출

In [ ]:
import sqlalchemy

In [ ]:
sqlalchemy.__version__
#!pip install sqlalchemy --upgrade

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update, delete, join, func

### 엔진 생성, 연결

In [ ]:
engine = create_engine('sqlite:///:memory:', echo = True) # 데이터 베이스 연결 엔진을 생성

In [ ]:
con = engine.connect()

### 테이블 생성

In [ ]:
meta = MetaData() #metadata 객체를 생성, 데이터 베이스의 스키마 생성 관리 가능

In [ ]:
Table('USER', meta, 
      Column('PK', Integer, primary_key=True),
      Column('NAME', Text, nullable = False))

In [ ]:
User = meta.tables['USER'] # user라는 테이블의 인스턴스

In [ ]:
print(User)
print(User.insert())

In [ ]:
meta.create_all(engine)

In [ ]:
meta.drop_all(engine) # 테이블 삭제

In [ ]:
meta.remove(User)
meta.tables

In [ ]:
meta.reflect(engine)
meta.tables #데이터 베이스 내의 모든 테이블 출력

In [ ]:
print(User.insert())
print(User.insert().values(PK= 1, NAME ="아무개")) # 프린트를 해보면 value를 지정하지 않은것도 똑같이 출력되는 것처럼 보인다.
print(User.insert().values(PK= 1, NAME ="아무개").compile().params) # 내가 지정한 파리미터 확인법

In [ ]:
con.execute(User.insert().values(PK= 1, NAME ="아무개"))

In [ ]:
con.execute(User.select()).fetchall()

In [ ]:
# 테이블 생성
Table('ADDRESS', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text),
      Column('FK', Integer, ForeignKey('USER.PK')))

In [ ]:
Table('TEMP', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text),
      Column('FK', Integer, nullable=False))

In [ ]:
User.columns['PK']

In [ ]:
meta.create_all(engine)

In [ ]:
con.execute(
    meta.tables['ADDRESS'].insert().values(PK=1, NAME ='주소', FK=1))

In [ ]:
con.execute(
    meta.tables['TEMP'].insert().values(PK=1, NAME ='가짜', FK=1)) # 위테이블과의 차이는 FK를 주었냐 안주었냐의 차이

In [ ]:
con.execute(
    select(meta.tables['USER'],
                   meta.tables["ADDRESS"],
                   meta.tables['TEMP'])
                   ).fetchall()

In [ ]:
con.execute(
    select(
        meta.tables['USER'].columns['PK'],
        meta.tables["ADDRESS"].c.NAME,
        meta.tables['TEMP'].c.PK
        )).fetchall()

In [ ]:
USER = meta.tables['USER']
ADDR = meta.tables['ADDRESS']
TEMP = meta.tables['TEMP']

In [ ]:
print(USER.select().where(USER.c.PK == 1))
print(USER.c.PK == TEMP.c.FK)
print(USER.join(ADDR))
print(USER.join(TEMP, USER.c.PK == TEMP.c.FK))

1030~

In [ ]:
from sqlalchemy.sql import or_, and_, between

In [ ]:
print(or_(USER.c.PK==1, USER.c.PK ==2))

In [ ]:
print(USER.c.NAME.like("%어쩌고%"))

In [ ]:
print(USER.select().where(USER.c.NAME.like('%어쩌고%')).\
    group_by(USER.c.PK).
    order_by(USER.c.PK))

In [ ]:
# and, or 를 사용하는 법
print((USER.c.NAME == "abcd") & (USER.c.PK == 1) | (USER.c.PK == 2)) 

In [ ]:
#테이블 간 join
print(USER.join(ADDR))

In [ ]:
# select_from을 사용해서 join => join한 테이블을 다시 사용할 떄 유리
print(select().select_from(USER.join(ADDR)))

### 예제

좋아하는 노래, 앨범, 가수, 장르에 관한 ER 모델
- Artist
- Album
- Genre
- Track

In [ ]:
meta.clear()

In [ ]:
engine = create_engine('sqlite:///:memory:', echo = True)

In [ ]:
con = engine.connect()

In [ ]:
meta = MetaData()

In [ ]:
# 4개의 테이블 만들기
# A: 노래
Table('T_A', meta,
        Column('PK', Integer, primary_key =True),
        Column('NAME', Text, nullable =False))
# B : 앨범
Table('T_B', meta,
        Column('PK', Integer, primary_key =True),
        Column('NAME', Text, nullable =False),
        Column('FK', None, ForeignKey('T_A.PK')))
# C : 가수
Table('T_C', meta,
        Column('PK', Integer, primary_key =True),
        Column('NAME', Text, nullable =False))
# D :곡
Table('T_D', meta,
    Column('PK', Integer, primary_key =True),
    Column('NAME', Text, nullable =False),
    Column('FK_1', None, ForeignKey('T_B.PK')),
    Column('FK_2', Integer, nullable =False))


In [ ]:
len(meta.tables)

In [ ]:
meta.create_all(engine)

In [ ]:
# 가수 4명을 테이블 A에 넣기
for name in ['가수1', '가수2', '가수3', '가수4']:
    con.execute(meta.tables['T_A'].insert().values(NAME=name))

In [ ]:
# 테이블 이름 다시 지정
A = meta.tables['T_A']
B = meta.tables['T_B']
C = meta.tables['T_C']
D = meta.tables['T_D']

In [ ]:
for key in ["1", "2", "3", "4"]:
    for name in ['앨범1', '앨범2']:
        fk = con.execute(select(A.c.PK).where(
            A.c.NAME.like('%' + key))).fetchone()[0]
        con.execute(B.insert().values(
            NAME=f'가수{key}_{name}', FK=fk))

In [ ]:
 con.execute(select(A.c.NAME, B.c.NAME).join(B)).fetchall()

In [ ]:
for name in ['장르1', '장르2', '장르3', '장르4']:
    con.execute(meta.tables['T_C'].insert().values(NAME=name))

In [ ]:
# 앨범 별로 곡을 네 곡 씩 넣기

for row in con.execute(select(B.c.PK)).fetchall():
    for key in ["1", "2", "3", "4"]:
        for name in ['노래1', '노래2', '노래3', '노래4']:
         fk = con.execute(select(C.c.PK).where(
            A.c.NAME.like('%' + key))).fetchone()[0]
         con.execute(D.insert().values(
            NAME=f'{row[0]}앨범_장르{key}_{name}', 
            FK_1= row[0],
            FK_2=fk))

In [ ]:
 con.execute(select(D.c.NAME)).fetchall()

In [ ]:
# sql의 function
# func.count, func.upper 등 func. 뒤에 함수를 불러올 수 있다.

In [ ]:
con.execute(select(func.CURRENT_TIMESTAMP())) #현재 시간 출력

In [ ]:
con.execute(select(func.sum(A.c.PK))).fetchall()

In [ ]:
# 각 테이블에 몇개가 있는지 확인 가능 
con.execute(select(func.count(A.c.PK))).fetchall(),\ 
con.execute(select(func.count(B.c.PK))).fetchall(),\
con.execute(select(func.count(C.c.PK))).fetchall(),\
con.execute(select(func.count(D.c.PK))).fetchall()

#,\을 사용하면 출력 한번에

In [ ]:
print(select(A.c.NAME, B.c.NAME).where(A.c.PK == B.c.FK))
con.execute(select(A.c.NAME, B.c.NAME).where(A.c.PK == B.c.FK)).fetchall()

In [ ]:
print(select(A.c.NAME, B.c.NAME).join(B))
print(select(A.c.NAME, B.c.NAME).join(B, A.c.PK == B.c.FK))
con.execute(select(A.c.NAME, B.c.NAME).join(B, A.c.PK == B.c.FK)).fetchall()


In [ ]:
print(select(A.c.NAME, B.c.NAME).select_from(A.join(B)))
con.execute(select(A.c.NAME, B.c.NAME).select_from(A.join(B))).fetchall()

### 1140

In [ ]:
### 가수-앨범-노래
print(select(A.c.NAME, B.c.NAME, D.c.NAME).select_from(B.join(D)))
con.execute(select(A.c.NAME, B.c.NAME, D.c.NAME).\
            select_from(A.join(B.join(D)))).fetchall()

In [ ]:
# 앨범을 기준으로 그룹하여 노래 개수 찾기.
con.execute(select(A.c.NAME, B.c.NAME, func.count(D.c.NAME)).\
    select_from(A.join(B.join(D))).\
        group_by(B.c.PK)).fetchall()

In [ ]:
#가수 이름에 3이 들어간 가수의 앨범과 곡의 개수 찾기
con.execute(select(A.c.NAME, B.c.NAME, func.count(D.c.NAME)).\
    select_from(A.join(B.join(D))).\
    where(A.c.NAME.like('%3')).\
    group_by(B.c.PK)).fetchall()


일을 시킬 대상을 구별하라(?)

존재하는 테이블을 덮어씌우는 옵션
- Table(~, extend_existing = True)

In [ ]:
# A에 입력
con.execute(
    insert(A).values(
        [{'NAME' : '아무개1'}, {'NAME' : '아무개2'}]
    )
)

In [ ]:
# A, B, C, D 테이블 모두 섞기
print(select(A.c.NAME, B.c.NAME, C.c.NAME, D.c.NAME).\
    select_from(A.join(B).join(D).join(C, C.c.PK==D.c.FK_2)))

In [ ]:
con.execute(select(D.c.NAME).select_from(D)).fetchall()

In [ ]:
con.execute(
    select(A.c.NAME, B.c.NAME, C.c.NAME, D.c.NAME).\
    select_from(A.join(B).join(D).join(C, C.c.PK==D.c.FK_2))).fetchall()

## 어제 했던거 해보기

In [ ]:
engine = create_engine('sqlite:///:memory:', echo = True)

In [ ]:
con = engine.connect()

In [ ]:
meta= MetaData()

In [ ]:
meta.create_all(engine)

In [ ]:
meta.tables

In [ ]:
Table('POST', meta,
    Column('PK', Integer, Primary_key = True),
    Column('CONTENT', Text),
    extend_existing = True
    )

Table('HASHTAG', meta,
    Column('PK', Integer, Primary_key = True),
    Column('NAME', Text, nullable = False),
    Column('COUNT', Integer, server_default = '0'),
    extend_existing = True
    )

Table('POSTTAG', meta,
    Column('PK', Integer, Primary_key = True),
    Column('PFK', None, ForeignKey('POST.PK')),
    Column('HFK', Integer, ForeignKey('HASHTAG.PK')),
    extend_existing = True
    )

In [ ]:
meta.create_all(engine)

In [ ]:
A = meta.tables['POST']
B = meta.tables['HASHTAG']
C = meta.tables['POSTTAG']

In [ ]:
con.execute(B.insert().values(NAME = 'A'))

In [ ]:
print(B.select().where(B.c.NAME == 'ORM'))
len(con.execute(B.select().where(B.c.NAME == 'ORM')).fetchall())

In [ ]:
key = 'ORM'
size = con.execute(select(func.count(B.c.PK)).where(B.c.NAME == key)).fetchone()[0]

if size == 0 :
    con.execute(B.insert().values(NAME =key))

con.execute(select(B.c.PK).where(B.c.NAME==key)).fetchone()[0]

직접 만들기 (?)
1. POST에 본문 INSERT
2. INSERT된 PK
3. 위 코드를 이용해 해쉬태그 PK 가져오기
4. POSTTAG에 POST.PK와 HASHTAG.PK를 insert
5. HASHTAG COUNT +1, PK = HASHTAG

In [ ]:
#1
con.execute(A.insert().values(PK =1, CONTENT = "data")).fetchall()